In [16]:
import numpy as np
import pandas as pd
from math import sqrt
import pickle
import time
import math
from math import sin
from math import cos
from math import pi
from minimax_tilting_sampler import TruncatedMVN # for truncated MVNs
import generate_data
import reg_dml
import statsmodels.api as sm
from doubleml import DoubleMLData
from doubleml import DoubleMLPLR
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor
from sklearn.base import clone
from scipy import stats
from doubleml.datasets import make_plr_CCDDHNR2018
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
import multiprocessing as mp
import warnings
warnings.filterwarnings("ignore")


In [17]:
import doubleml
print(doubleml.__version__)

0.9.0


In [18]:
#Initial the paramter
mean_x=np.array([0.00968,0.737,0.019,0.00145,0.011])
b_x=8
b_w=np.array([0,0,0,0,22,0])

sigma_x=np.diag([1, 1, 1, 1,1])
sigma_ep=np.diag([1, 1, 1, 1,1])
age_min=18
age_max=91

n1=350
n2=1000

b_0=1
sd_xi=9 # sd for the error in main model

In [19]:
#Generate validation dataset (v) and main dataset (m)
v=generate_data.ge_right(n1,age_min,age_max,mean_x,sigma_x,sigma_ep)
m=generate_data.ge_main(n2,age_min,age_max,mean_x,b_0,b_x,b_w,sd_xi,sigma_x,sigma_ep,True)

In [20]:
#Use genearlized estimating equation (GEE) to estimate the parameters in measurement error model 
x_per_v=v[:,1]
x_nm_v=v[:,1+mean_x.shape[0]]
w1_per_v=v[:,0:mean_x.shape[0]]
w1_per_v=np.delete(w1_per_v, 1, axis=1)
w1_nm_v=v[:,(mean_x.shape[0]):(2*mean_x.shape[0])]
w1_nm_v=np.delete(w1_nm_v, 1, axis=1)
w2_v=v[:,[2*mean_x.shape[0],2*mean_x.shape[0]+1]]
gee_results=reg_dml.data_me_gee(x_per_v,x_nm_v,w1_per_v,w1_nm_v,w2_v)
gee_params=gee_results[0]
gee_var=gee_results[1]

In [21]:
#estimate the causal effect in main study
y_m=m[0]
w1_nm_m=m[1][:,(mean_x.shape[0]):(2*mean_x.shape[0])]
w1_nm_m=np.delete(w1_nm_m, 1, axis=1)
x_nm_m=m[1][:,1+mean_x.shape[0]]
w2_m=m[1][:,[2*mean_x.shape[0],2*mean_x.shape[0]+1]]
(x_pred_per,w1_pred_per,xw_nm_m)=reg_dml.gee_predict(x_nm_m,w1_nm_m,w2_m,gee_params)
w1_pred_per_w2=np.concatenate((w1_pred_per,w2_m),axis=1)
w1_x_pred_per=np.concatenate((x_pred_per.reshape(-1,1),w1_pred_per,w2_m),axis=1)
w1_x_pred_per_add_cons=sm.add_constant(w1_x_pred_per)
    
nm_xd=m[1][:,(mean_x.shape[0]):(2*mean_x.shape[0])]
nm_xd=np.array(sm.add_constant(nm_xd))
print("Total R square is",m[3],"\t","R square for X is",m[2],"\t","R square for g is",m[4])

w_per=m[1][:,np.concatenate([range(mean_x.shape[0]),[2*mean_x.shape[0]],[2*mean_x.shape[0]+1]],axis=0)]
w_per=np.delete(w_per, 1, axis=1)
x_per=m[1][:,1]
w_nm=np.concatenate([w1_nm_m,w2_m],axis=1)
x_nm=x_nm_m
data_per=list()
data_per.append((w_per,y_m,x_per))
data_nm=list()
data_nm.append((w_nm,y_m,x_nm_m))

data_pred = (w1_pred_per_w2,y_m,x_pred_per)


dml_lasso=reg_dml.dml_lasso(data_pred,b_x, True,True, w1_x_pred_per_add_cons, xw_nm_m,gee_var,gee_params)       
per_dml_lasso=reg_dml.dml_lasso(data_per[0],b_x, False,True, w1_x_pred_per_add_cons, xw_nm_m,gee_var,gee_params)  
nm_dml_lasso=reg_dml.dml_lasso(data_nm[0],b_x, False,True, w1_x_pred_per_add_cons, xw_nm_m,gee_var,gee_params)
results=np.concatenate((dml_lasso,per_dml_lasso, nm_dml_lasso),axis=None)    

Total R square is 0.9998446471642629 	 R square for X is 0.00017816603648242933 	 R square for g is 1.0004760580753294
ML_full_sample {'ml_l': {'d': [[{'alpha': np.float64(0.096)}, {'alpha': np.float64(0.096)}]]}, 'ml_m': {'d': [[{'alpha': np.float64(0.006)}, {'alpha': np.float64(0.006)}]]}, 'ml_g': {'d': [[{'alpha': np.float64(0.096)}, {'alpha': np.float64(0.096)}]]}}
ML_full_sample {'ml_l': {'d': [[{'alpha': np.float64(0.096)}, {'alpha': np.float64(0.096)}]]}, 'ml_m': {'d': [[{'alpha': np.float64(0.026000000000000002)}, {'alpha': np.float64(0.026000000000000002)}]]}, 'ml_g': {'d': [[{'alpha': np.float64(0.096)}, {'alpha': np.float64(0.096)}]]}}
ML_full_sample {'ml_l': {'d': [[{'alpha': np.float64(0.046)}, {'alpha': np.float64(0.046)}]]}, 'ml_m': {'d': [[{'alpha': np.float64(0.096)}, {'alpha': np.float64(0.096)}]]}, 'ml_g': {'d': [[{'alpha': np.float64(0.096)}, {'alpha': np.float64(0.096)}]]}}


In [22]:
#print the results
print("The bias of DML-RC is",dml_lasso[0],"with standard error is", dml_lasso[1])
print("The bias of DML-True is",per_dml_lasso[0],"with standard error is", per_dml_lasso[1])
print("The bias of uncorrected DML is",nm_dml_lasso[0],"with standard error is", nm_dml_lasso[1])

The bias of DML-RC is -0.48757669026867045 with standard error is 0.8452148430776865
The bias of DML-True is -0.18866176723728945 with standard error is 0.35321485645661344
The bias of uncorrected DML is -4.926471082931898 with standard error is 0.349276090875872
